Setup for send - receive test with 2 nodes

- Pico on COM15 = transmitter
- Pico-W on COM4 = receiver

use micropython magig and mpremote to connect


In [13]:
devices = %mpy --list
ports = devices.fields(0)

for port in ports:
    s = %mpy --info --select {port}
    print(f" {s['serial_port']:<7}: {s['board']}")
    
PICO = 'COM15'
PICO_W = 'COM4'

SENDER = PICO
RECEIVER = PICO_W

%mpy --select {SENDER} --reset
%mpy --select {RECEIVER} --reset

 COM4   : Raspberry Pi Pico W with RP2040
 COM15  : Raspberry Pi Pico with RP2040
True
True


In [14]:
# !mpremote connect {SENDER} cp -r . :
# !mpremote connect {RECEIVER} cp -r . :

In [15]:
%mpy --select {SENDER} --timeout 1 import send_universe

WARNING  | micropython_magic.interactive:timed_out:121 - Command mpremote connect COM15 resume  exec "import send_universe" timed out after 1.0 seconds


['sent universe 123',
 'sent universe 124',
 'sent universe 125',
 'sent universe 126',
 'sent universe 127',
 'sent universe 128',
 'sent universe 129',
 'sent universe 130',
 'sent universe 131',
 'sent universe 132',
 'sent universe 133',
 'sent universe 134',
 'sent universe 135',
 'sent universe 136',
 'sent universe 137',
 'sent universe 138',
 'sent universe 139',
 'sent universe 140']

In [16]:
# %%micropython --select {RECEIVER}
from rp2 import PIO, StateMachine, asm_pio
from machine import Pin

led = Pin("LED", Pin.OUT)
# remove all programs from both PIOs
for n in (0, 1):
    pio = PIO(n)
    led.toggle()
    pio.remove_program()

led.on()

In [17]:
# %%micropython --select {RECEIVER}

# pin_dmx_tx = Pin(15, Pin.OUT)  # send data to the DMX bus
pin_dmx_rx = Pin(14, Pin.IN)  # receive data from the DMX bus
max485_send = Pin(12, Pin.OUT, Pin.PULL_DOWN)  # switch send/receive for the MAX485 chip

p0 = Pin(0, Pin.OUT)  # debug pin
p1 = Pin(1, Pin.OUT)  # debug pin

In [25]:
# %%micropython --select {RECEIVER}

from typing_extensions import TYPE_CHECKING  # type: ignore

if TYPE_CHECKING:
    from rp2.asm_pio import *


max485_send.off()


#  from pio_dmx import dmx_receive, dmx_send

# fmt: off
@asm_pio(
        in_shiftdir=PIO.SHIFT_RIGHT,
        sideset_init=(PIO.OUT_LOW,PIO.OUT_LOW)
        
)
def dmx_receive():
    """PIO program to receive a DMX Universe frame of 512 channels."""
    # Constants
    dmx_bit = 4  # As DMX has a baudrate of 250.000kBaud, a single bit is 4us

    # Break loop
    # Receiver DMX break signal is 88us, so we need to loop 22 times to get 88us
    label("break_reset")
    set(x, 22).side(0b11)                          # 0

    label("break_loop")                 # BREAK = low for 88us
    jmp(pin, "break_reset")             # 1 | Go back to start if pin goes high during BREAK
    jmp(x_dec, "break_loop")        [1] # 2 | wait until BREAK time over (22 loops * 4us = 88us)
    
    wait(1, pin, 0).side(0b01)                     # 3 | wait for the Mark-After-Break (MAB)

    # Data loop
    # First start bit   - no need to detect end of frame
    label("byteloop")                    # Start of a byte
    wait(0, pin, 0)                 [1] # 4 | Wait for START bit (low) + 1+1us - measure halfway through the bit
    # Delay [2] measures just before the middle of the first bit
    set(x, 7)                       [3] # 5 | 7 more bit;  skip to (about) halfway first bit
    # Delay [3] measures just after the middle of the first bit

    label("bitloop")
    in_(pins, 1).side(0b10)                       # 6 Shift data bit into ISR
    jmp(x_dec, "bitloop").side(0b00)           [2] # 7 Loop 8 times, each loop iteration is 4us

    # Stop bits
    wait(1, pin, 0)                     # 8 Wait for pin to go high for stop bit-1
    nop().side(0b11)               [1]  # 9 Wait for pin to go high for stop bit-2
    # wait(1, pin, 0)                      # 9 Wait for pin to go high for stop bit-2
    #  in_(null, 24 )                      
    # TODO check if STOP bits are at 8us long
    # if longer than 8 us then we are at the end of the frame  MARK Time after Slot 512
    # this can be up to  1 second - which may be too long for a PIO program to count
    push().side(0b10)                           # 9
    jmp("byteloop")
# fmt: on



# create a state machine for receiving DMX data
machine_nr = 1


sm_dmx_rx = StateMachine(
    machine_nr,
    dmx_receive,
    freq=1_000_000,
    in_base=pin_dmx_rx,
    jmp_pin=pin_dmx_rx,
    sideset_base=p0,
)


sm_dmx_rx.active(0)


sm_dmx_rx.restart()

In [19]:
# This is a Q&D way to get the PIO opcodes from the PIO assembler using an attached rp2 with micropython
from array import array
x = %mpy print(dmx_receive)
asm_pio = eval(str(x[0])) # first line of the output is the PIO program, with parameters
assembled = asm_pio[0] # just the opcodes 
print(f"{len(assembled)} opcodes assembled")

", ".join([hex(opcode) for opcode in assembled])

12 opcodes assembled


'0xfc3d, 0xc0, 0x141, 0x34a0, 0x2120, 0xe227, 0x5801, 0x1246, 0x20a0, 0xbd42, 0x9820, 0x4'

In [26]:
# %%micropython --select {RECEIVER}
import time

size = 512
from array import array

universe = array("B", [0] + [0] * (size))  # 1 start code + 512 channels


# clear the fifo
sm_dmx_rx.active(0)
while sm_dmx_rx.rx_fifo():
    _ = sm_dmx_rx.get()
sm_dmx_rx.restart()

sm_dmx_rx.active(1)
sm_dmx_rx.get(universe, 24)  # get the first byte, which is the start code
sm_dmx_rx.active(0)
# print(f"{r[:10]=}")
print(universe[:10])

array('B', [143, 1, 2, 3, 4, 5, 6, 7, 8, 9])


In [27]:
# %%micropython --select {RECEIVER}
import utime


def timed_function(f, *args, **kwargs):
    "enables any function or method to be timed by adding an @timed_function decorator"
    myname = str(f).split(" ")[1]

    def new_func(*args, **kwargs):
        t = utime.ticks_us()
        result = f(*args, **kwargs)
        delta = utime.ticks_diff(utime.ticks_us(), t)
        print("Function {} Time = {:6.3f}ms".format(myname, delta / 1000))
        return result

    return new_func


# @timed_function
def rx_universe(universe):
    """
    read a received DMX universe into a bytearray
    Slow and careful version - on full speed will get every other DMX-Frame
    """
    sm_dmx_rx.active(0)
    # clear the fifo
    while sm_dmx_rx.rx_fifo():
        _ = sm_dmx_rx.get()
    sm_dmx_rx.restart()
    sm_dmx_rx.active(1)
    sm_dmx_rx.get(universe, 24)  # get the first byte, which is the start code
    sm_dmx_rx.active(0)
    return universe


rx_universe(universe)
print(universe[:10])

array('B', [162, 1, 2, 3, 4, 5, 6, 7, 8, 9])


In [33]:
# %%micropython --select {RECEIVER}
led = Pin("LED", Pin.OUT)


def rx_universe(universe):
    # sm_dmx_rx.active(0)
    # clear the fifo
    # print(f"{sm_dmx_rx.rx_fifo()=}")
    # while sm_dmx_rx.rx_fifo():
    #     _ = sm_dmx_rx.get()
    sm_dmx_rx.restart()
    sm_dmx_rx.active(1)
    sm_dmx_rx.get(universe, 24)  # get the first byte, which is the start code
    # led.toggle()
    # sm_dmx_rx.active(0)
    return universe


# clear the fifo
sm_dmx_rx.active(0)
print(f"{sm_dmx_rx.rx_fifo()=}")
while sm_dmx_rx.rx_fifo():
    _ = sm_dmx_rx.get()

l = []
v = []
for i in range(50):
    rx_universe(universe)
    l.append(universe[0])
    v.append(universe[501])
    # print(universe[:10])

print("Missing frames:", end=" ")
for i in range(1, len(l)):
    if l[i] - l[i - 1] > 1:
        print(l[i], end=" ")
print()

print(v)
print(f"{universe=}")
# sm_dmx_rx.active(0)

sm_dmx_rx.rx_fifo()=4
Missing frames: 
[245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245, 245]
universe=array('B', [30, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159

In [31]:
for i in range(1, len(l)):
    if l[i] - l[i - 1] > 1:
        print(l[i])

NameError: name 'l' is not defined